<a href="https://colab.research.google.com/github/mrachaba/Machine-Learning-Models/blob/main/Is_this_a_rose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNN for CIFAR 100 to tell whether a flower uploaded as input is a rose or not!**

*# Step 1: Load the CIFAR-100 dataset and filter, extract, convert labels to tensor flow in relevant classes*

In [59]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
tf.config.run_functions_eagerly(True)



In [60]:
(train_data, test_data), info = tfds.load('cifar100', split=('train', 'test'), with_info=True)

#assume the label for roses is 70
rose_class_index =70
train_data

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


<_PrefetchDataset element_spec={'coarse_label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [64]:
def extract_CIFAR100_to_dataframe(dataset, bool_param):
    records = []

    # Function to process dataset elements
    def process_element(element):
        record = {
            'coarse_label': element['coarse_label'].numpy(),
            'id': element['id'].numpy(),
            'image': element['image'].numpy(),
            'label': element['label'].numpy()
        }
        return record
    def process_element_without_numpy(element):
        record = {
          'coarse_label': element['coarse_label'],  # No need to call numpy() here
          'id': element['id'],                      # No need to call numpy() here
          'image': element['image'],                # No need to call numpy() here
          'label': element['label']                 # No need to call numpy() here
          }
        return record

    if bool_param == True:
      # Convert the dataset to a numpy array
      dataset_np = np.array(list(dataset.as_numpy_iterator()))
      records = map(process_element_without_numpy, dataset_np)
    else:
      # Extract elements and process them
      for element in dataset:
        records.append(process_element(element))

    # Convert list of records to DataFrame
    df = pd.DataFrame(records)

    return df



In [65]:
test_1 = extract_CIFAR100_to_dataframe(train_data, False)
test_1.head()

,coarse_label,id,image,label
0,12,b'train_16399',"[[[151, 154, 145], [144, 147, 135], [141, 143,...",66
1,10,b'train_01680',"[[[250, 250, 250], [248, 248, 248], [245, 245,...",23
2,5,b'train_47917',"[[[2, 6, 2], [6, 8, 2], [11, 9, 1], [22, 15, 3...",40
3,5,b'train_17307',"[[[247, 247, 247], [244, 244, 244], [245, 245,...",39
4,7,b'train_27051',"[[[160, 151, 143], [148, 136, 128], [155, 141,...",7


In [66]:
test_2 = extract_CIFAR100_to_dataframe(train_data, True)
test_2.head()

,coarse_label,id,image,label
0,12,b'train_16399',"[[[151, 154, 145], [144, 147, 135], [141, 143,...",66
1,10,b'train_01680',"[[[250, 250, 250], [248, 248, 248], [245, 245,...",23
2,5,b'train_47917',"[[[2, 6, 2], [6, 8, 2], [11, 9, 1], [22, 15, 3...",40
3,5,b'train_17307',"[[[247, 247, 247], [244, 244, 244], [245, 245,...",39
4,7,b'train_27051',"[[[160, 151, 143], [148, 136, 128], [155, 141,...",7


In [ ]:
# Filter images labelled as roses
rose_train_data = [(element['image'], element['label']) for element in train_data if element['label'] == rose_class_index]
rose_test_data = [(element['image'], element['label']) for element in test_data if element['label'] == rose_class_index]

In [ ]:
# Extract labels for rose images and convert rose labels to TensorFlow tensors
rose_train_images = tf.convert_to_tensor([image for image, _ in rose_train_data])
rose_test_images = tf.convert_to_tensor([image for image, _ in rose_test_data])
rose_train_labels = tf.convert_to_tensor([label == rose_class_index for _, label in rose_train_data])  # Binary labels
rose_test_labels = tf.convert_to_tensor([label == rose_class_index for _, label in rose_test_data])  # Binary labels

In [ ]:
# Define your CNN model
model = tf.keras.Sequential([
    # Add convolutional layers, pooling layers, and fully connected layers as needed
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(rose_train_images, rose_train_labels, epochs=9, validation_data=(rose_test_images, rose_test_labels))

Epoch 1/9
16/16 [==============================] - 1s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/9
16/16 [==============================] - 1s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/9
16/16 [==============================] - 1s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/9
16/16 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/9
16/16 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/9
16/16 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/9
16/16 [==============================] - 1s 33ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(rose_test_images, rose_test_labels)

# Print the test loss and accuracy
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


4/4 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Loss: 0.0
Test Accuracy: 1.0


In [4]:
data = range(0,1000)

In [12]:
 range(7-2)

range(0, 5)

In [33]:
def fibonacci_series(x):

  """ This is a fibonocci series converter. That function takes a 1D range of 0 to n
   x: is a range variable with max n and it is an int datatype
   returns fibonocci series for the given range"""

  fib_series = [0,1] # initializing the fibonacci series with 0, 1
  if (x == 0):
    return fib_series[0]
  for i in range(2, x+1):
    #fib_series[i] + fib_series[i+1] == sum
    fib_series.append(fib_series[i-1] + fib_series[i-2])
  return fib_series


In [34]:
print(fibonacci_series(1))

[0, 1]
